<table width="100%" border="0">
<tr>
<td width="50%" bgcolor="#FFF" border="1">
<a href="https://www.health-samurai.io/">
<img src="images/health-samurai.png" alt="Health Samurai" align="left" />
</a>
</td>
<td width="50%" bgcolor="#FFF">
<a href="http://beda.software/">
<img src="images/beda-software.png" alt="Beda.Software" align="right" />
</a>
</td>
</tr>
</table>

# Laboratory work #4. FHIR resource Appointment

Make an appointment for the patient (John Thompson) from the lab#1 to the practitioner selected  in the lab #3 for next Monday (09-16-2018).

## 1 Introduction

### 1.1 Import library

For the beginning we should import the `fhirpy` and `os` libraries.  
Also we import `pprint` helper function from local `utils.py` file. We'll use `pprint` to display some structures.

In [ ]:
import os
import fhirpy

from utils import pprint

### 1.2 Create an instance of connection

To load data from FHIR server we should instantiate `FHIRClient` class from `fhirpy` package.  
We pass `url` and `authorization` arguments from environment.

In [ ]:
client = fhirpy.AsyncFHIRClient(
    url=os.environ['BOX_URL'] + '/fhir', 
    authorization=os.environ['BOX_AUTHORIZATION'])

Now, we are able to operate with FHIR resources using `client`.

## 2 Find participants

**Exercise**: Find a patient whose name is `John Thompson`

In [ ]:
patient = await client.resources('Patient').search(name=['John', 'Thompson']).first()
patient

**Exercise**: Find the practitioner we've choosen before. Her name is `Kelly Smith`

In [ ]:
practitioner = await client.resources('Practitioner').search(
    # Write code here
).first()
practitioner

## 3 Select date

**Exercise**: Using `Schedule` [search parameters](https://www.hl7.org/fhir/schedule.html#search) find the schedule to use with practitioner

In [ ]:
schedule = await client.resources('Schedule').search(
    # Write code here
).first()
schedule

**Exercise**: For the schedule found choose a time slot at 11 am on Monday 16

In [ ]:
slot = await client.resources('Slot').search(
    schedule=schedule['id'],
    # Write code here
).first()
(slot.get('start'), slot.get('end'), slot.get('status'))

Mark selected time slot as `busy`

In [ ]:
slot['status'] = 'busy'
await slot.save()

## 4 Appointment

[Appointment](https://www.hl7.org/fhir/appointment.html) resources are used to provide information about a planned meeting (only one) that may be in the future or past.

`Appointment` can be considered as administrative only, and the `Encounter` is expected to have clinical implications. In general, it is expected that appointments will result in the creation of an `Encounter`. The encounter is typically created when the service starts, not when the patient arrives.

When Request/Response pattern is in use, a new `Appointment` is created with `status=proposed` or `status=pending` and the list of actors with status of `needs-action`. [AppointmentResponse](https://www.hl7.org/fhir/appointmentresponse.html) is then used by participants to respond with their acceptance (or not) to the appointment. Once all of the participants have replied, the appointment gets it's final status.

In an Emergency Room context, the appointment resource is replaced with `Encounter`.

To assign an `Appointment` to a specific date:
* Determine address details of the resource we want to schedule appointment with.
* Check available `Slot`s for that time.
* Create new `Appointment` resource with `Appointment.status=proposed` and `status=needs-action` for all `Appointment.participants`.
* Wait for `Appointment.status` updates for approved or rejected appointments.

We are already agreed on the time slot. All of the addressing information is provided by the `Schedule` the time slot is booked for.

Let's create a new `Appointment`

In [ ]:
# All of the actors from Schedule
participants = [{'actor': actor, 'status': 'needs-action'} for actor in schedule['actor']]

# Plus the patient
participants += [{'actor': patient.to_reference(), 'status': 'accepted'}]

appointment = client.resource(
    'Appointment',
    id='appointment_for_john_thompson',
    status='proposed',
    start=slot['start'],
    end=slot['end'],
    slot=[slot],
    participant=participants
)
await appointment.save()

pprint(appointment)

## 4 Find appointments

**Exercise**: Using `Appointment` [search parameters](https://www.hl7.org/fhir/appointment.html#search) get a list of appointments for the patient.

In [ ]:
appointments = await client.resources('Appointment').search(
    # Write code here
).fetch()
[(ap.get('start'), ap.get('end'), ap.get('status')) for ap in appointments]

## Summary

In this laboratory work we've discussed:
* How to book a time slot
* How to create an `Appointment`
* How to search for `Appointments`

<table width="100%" border="0">
<tr>
<td width="50%" bgcolor="#FFF" border="1">
<a href="https://www.health-samurai.io/">
<img src="images/health-samurai.png" alt="Health Samurai" align="left" />
</a>
</td>
<td width="50%" bgcolor="#FFF">
<a href="http://beda.software/">
<img src="images/beda-software.png" alt="Beda.Software" align="right" />
</a>
</td>
</tr>
</table>